<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br><h2>Script 03 | Feature Engineering</h2>
<br>
Written by Chase Kusterer<br>
<a href="https://github.com/chase-kusterer">GitHub</a> | <a href="https://www.linkedin.com/in/kusterer/">LinkedIn</a>
<br><br><br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h2>Introducing Feature Engineering</h2><br>
Oftentimes, the data we have available was not originally intended to predict the phenomenon we are interested in. Analysts are rarely the authors of their data, and it is also rare that they are able to access good quality documentation on its collection method(s), assumptions, or original purpose. Even if they <em>were</em> the authors, we learned the challenges associated with this in <em>Business Analysis with Structured Data</em>. Long story short: we, as analysts, need <strong>to make efforts to restructure the data so that it is better focused on our phenomenon of interest</strong>. This is the purpose of <strong>feature engineering</strong>.
<br><br>
In our previous script when we developed flag features to represent missing values, we were performing feature engineering. In other words, we created new features based on existing ones. The missing value flags also aligned with <strong>two important principles of feature engineering</strong>: 1) there was <strong>strong evidence in the data</strong> to justify their creation, and 2) there was a <strong>strong link to something practical</strong> in the real world (the reason why the values were missing).
<br><br><br>
<h2>Part I: Developing Trend-Based Features</h2><br>
In this section, we will expand our knowledge of flag features by engineering binary trend-based features. Our goal is to identify where a feature's trend changes in terms of its relationship with what we are trying to predict. This may improve our predictions while helping to generate insights.<br><br>
Let's start by importing libraries and the data.

In [ ]:
# importing libraries
import pandas as pd                   # data science essentials
import matplotlib.pyplot as plt       # essential graphical output
import seaborn as sns                 # enhanced graphical output
import numpy as np                    # mathematical essentials
import statsmodels.formula.api as smf # regression modeling
import warnings                       # warnings


# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# suppressing warnings
warnings.filterwarnings(action = 'ignore')


# specifying file path
file = './datasets/ames_transformed.xlsx'


# reading the file into Python
housing = pd.read_excel(io     = file,
                        header = 0   )


# code will not produce an output

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

Run the code below to generate a scatter plot of the relationship between <em>Second_Flr_SF</em> and <em>log_Sale_Price</em>.

In [ ]:
# developing a scatterplot
sns.scatterplot(x     = housing['Second_Flr_SF'],
                y     = housing['log_Sale_Price'], # unskewed Sale_Price
                color = 'royalblue')


# title and labels
plt.title(label   = """
Relationship between Logarithmic Sale Price and
Second Floor Square Footage""")
plt.xlabel(xlabel = 'Second Floor SF')
plt.ylabel(ylabel = 'Sale Price (log)')


# cleaning up the layout, saving the figures, and displaying the results
plt.tight_layout()
plt.show()

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

As can be observed, several observations have a second floor square footage of zero. Put simply, these properties do not have a second floor. This is something we can take advantage of with a binary trend-based feature, enabling us to analyze the effect second floors have on sale prices. Before engineering, let's analyze a few other x-features.

<strong>a)</strong> Complete the code below to develop a scatter plot between each x-feature and <em>log_Sale_Price</em>.
Reuse the code cell as many times as needed.

In [ ]:
# developing a scatterplot
sns.scatterplot(_____)


# title and labels
plt.title(label   = _____)
plt.xlabel(xlabel = _____)
plt.ylabel(ylabel = _____)


# cleaning up layout and rendering visual
plt.tight_layout()
plt.show()

In [ ]:
# developing a scatterplot
sns.scatterplot(x     = housing['Mas_Vnr_Area'],
                y     = housing['log_Sale_Price'], # unskewed Sale_Price
                color = 'royalblue')


# title and labels
plt.title(label   = """
Relationship between Logarithmic Sale Price and
Masonic Veneer Area""")
plt.xlabel(xlabel = 'Masonic Veneer')
plt.ylabel(ylabel = 'Sale Price (log)')


# cleaning up layout and rendering visual
plt.tight_layout()
plt.show()

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
From the scatter plots above, it is evident that several features have a high frequency of zero values. Let's explore further.
<br><br><br>
<strong>Basic Rule for Flag-Based Feature Engineering</strong>

* At least 100 observations must be on both sides of the flag (1s and 0s)

<br>
<strong>b)</strong> Use <em>len(&nbsp;)</em> and subsetting in the code below to analyze features that have a high frequency of zero values.

In [ ]:
# counting the number of zeroes for 
bsmt_zeroes   = ____(housing['Total_Bsmt_SF'][____]) # Total_Bsmt_SF
sf_zeroes     = ____ # Second_Flr_SF
garage_zeroes = ____ # Garage_Area
mas_zeroes    = ____ # Mas_Vnr_Area
porch_zeroes  = ____ # Porch_Area


## the table below has been coded for you ##

# printing a table of the results
print(f"""
                 No\t\tYes
               ---------------------
Basement       | {bsmt_zeroes}\t\t{len(housing) - bsmt_zeroes}
Second Floor   | {sf_zeroes}\t\t{len(housing) - sf_zeroes}
Garage         | {garage_zeroes}\t\t{len(housing) - garage_zeroes}
Masonic Veneer | {mas_zeroes}\t\t{len(housing) - mas_zeroes}
Porch          | {porch_zeroes}\t\t{len(housing) - porch_zeroes}
""")

In [ ]:
# counting the number of zeroes
bsmt_zeroes   = len(housing['Total_Bsmt_SF'][housing['Total_Bsmt_SF'] == 0])
sf_zeroes     = len(housing['Second_Flr_SF'][housing['Second_Flr_SF'] == 0])
garage_zeroes = len(housing['Garage_Area'][housing['Garage_Area'] == 0])
mas_zeroes    = len(housing['Mas_Vnr_Area'][housing['Mas_Vnr_Area'] == 0])
porch_zeroes  = len(housing['Porch_Area'][housing['Porch_Area'] == 0])


## the table below has been coded for you ##

# printing a table of the results
print(f"""
                 No\t\tYes
               ---------------------
Basement       | {bsmt_zeroes}\t\t{len(housing) - bsmt_zeroes}
Second Floor   | {sf_zeroes}\t\t{len(housing) - sf_zeroes}
Garage         | {garage_zeroes}\t\t{len(housing) - garage_zeroes}
Masonic Veneer | {mas_zeroes}\t\t{len(housing) - mas_zeroes}
Porch          | {porch_zeroes}\t\t{len(housing) - porch_zeroes}
""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<strong>c)</strong> Based on the results above, engineer binary trend-based features for all variables that have at least 100 observations in both the 'yes' and 'no' columns.

In [ ]:
# placeholder variables
housing['has_Second_Flr'] = 0
housing['has_Garage']     = 0
housing['has_Mas_Vnr']    = 0
housing['has_Porch']      = 0


# iterating over each original column to
# change values in the new feature columns
for index, value in ____.____():


    # Second_Flr_SF
    if housing.loc[index, ____] > 0:
        housing.loc[index, 'has_Second_Flr'] = 1
        
        
    # Garage_Area
    if housing.loc[index, ____] > 0:
        housing.loc[index, 'has_Garage'] = 1
        
        
    # Mas_Vnr_Area
    if housing.loc[index, ____] > 0:
        housing.loc[index, 'has_Mas_Vnr'] = 1
        
        
    # Porch_Area
    if housing.loc[index, ____] > 0:
        housing.loc[index, 'has_Porch'] = 1
        
        
## this code will not produce an output ##

In [ ]:
# placeholder variables
housing['has_Second_Flr'] = 0
housing['has_Garage']     = 0
housing['has_Mas_Vnr']    = 0
housing['has_Porch']      = 0


for index, value in housing.iterrows():
    

    # Second_Flr_SF
    if housing.loc[index, 'Second_Flr_SF'] > 0:
        housing.loc[index, 'has_Second_Flr'] = 1
        
        
    # Garage_Area
    if housing.loc[index, 'Garage_Area'] > 0:
        housing.loc[index, 'has_Garage'] = 1
        
        
    # Mas_Vnr_Area
    if housing.loc[index, 'Mas_Vnr_Area'] > 0:
        housing.loc[index, 'has_Mas_Vnr'] = 1
        
        
    # Porch_Area
    if housing.loc[index, 'Porch_Area'] > 0:
        housing.loc[index, 'has_Porch'] = 1
        
        
## this code will not produce an output ##

<br>

In [ ]:
# checking results
housing[  ['has_Second_Flr', 'has_Garage', 'has_Mas_Vnr', 'has_Porch']  ].head(n = 5)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

A new feature idea was discovered while exploring <em>Lot_Area</em> in an earlier script. Run the code below to engineer this feature into the housing dataset. Note that this feature is a <strong>ratio</strong> between a the area of property's buildings and its land.

In [ ]:
## ratio between buildings and lot area ##

# summing building features
buildings = housing[["Gr_Liv_Area", "Garage_Area", "Porch_Area"]].sum(axis = 1)


# ratio between buildings and lot area
ratio_building_lot = (buildings / housing["Lot_Area"])


# augmenting new feature to housing dataset
housing["ratio_building_lot"] = ratio_building_lot


# checking results
housing["ratio_building_lot"].head(n = 5)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

Let's check the correlations between the newly-engineered features with <em>Sale_Price</em> and <em>log_Sale_Price</em>.

In [ ]:
# developing a small correlation matrix
new_corr = housing.corr(method = 'pearson').round(decimals = 2)


# checking the correlations of the newly-created variables with Sale_Price
new_corr.loc[ ['has_Second_Flr', 'has_Garage', 'has_Mas_Vnr',
               'has_Porch', 'ratio_building_lot'],
              ['Sale_Price', 'log_Sale_Price'] ].sort_values(by = 'Sale_Price',
                                                             ascending = False)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h2>Part II: Feature Engineering with Interval and Count Data</h2><br>
Interval and count data for the Ames housing dataset can be found in <em>ames_non_continuous.xlsx</em>. Note that this file contains two sheets: <em>interval_count</em> and <em>categorical_discrete</em>. Remember, each <strong>classification</strong> of data (continuous, interval, etc.) comes with its own assumptions and philosophy of mathematics. This is the reason why they are analyzed separately. Organization is key to conducting a good analysis!
<br><br>
<strong>a)</strong> Complete the code below to import the <em>interval_count</em> sheet of <em>ames_non_continuous.xlsx</em>.

In [ ]:
# importing interval and count data
file = _____

housing_2 = pd.read_excel(io         = file,
                          header     = 0,
                          sheet_name = _____)


# merging the two datasets
housing = housing._____(right = _____,
                        on    = _____,
                        how   = _____)


# checking results
housing_2.head(n = 5)

In [ ]:
# documentation: help(pd.DataFrame.merge)

# importing interval and count data
file = './datasets/ames_non_continuous.xlsx'

housing_2 = pd.read_excel(io         = file,
                          header     = 0,
                          sheet_name = 'interval_count')


# checking results
housing_2.head(n = 5)

<br>

In [ ]:
# general info about the dataset
housing_2.info(verbose = True)

<br>

In [ ]:
# descriptive statistics
housing_2.describe(include = 'number').round(decimals = 2)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Before moving forward, let's join the two housing datasets.

In [ ]:
# documentation for the join method
help(housing.join)

<br>

In [ ]:
# setting property_id as index for both DataFrames
housing   = housing.set_index(keys = 'property_id')
housing_2 = housing_2.set_index(keys = 'property_id')


# left join to ensure retention of original data
housing = housing.join(other    = housing_2,
                       how      = 'left',
                       validate = 'one_to_one')


# checking results
housing.columns

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Scatter plots are not as useful when working with interval or count data. It is a better practice is to use box plots as this type of visual aggregates a distribution based on its quartiles. Also note that it is not a good practice to transform (logarithmic, etc.) interval or count data as interpretation becomes incredibly challenging. We will learn new techniques for these types of data.

In [ ]:
# scatterplot - orig ovr_qual and price
sns.scatterplot(x    = 'Overall_Qual',
                y    = 'Sale_Price' ,
                data = housing)


# titles and axis labels
plt.title(label   = 'Scatterplot with Interval Data')
plt.xlabel(xlabel = 'Overall Quality')
plt.ylabel(ylabel = 'Sale Price')


# displaying the plot
plt.show()

In [ ]:
# boxplot - orig ovr_qual and price
sns.boxplot(x    = 'Overall_Qual',
            y    = 'Sale_Price',
            data = housing)


# titles and axis labels
plt.title(label   = 'Boxplot with Interval Data')
plt.xlabel(xlabel = 'Overall Quality')
plt.ylabel(ylabel = 'Sale Price')


# displaying the plot
plt.show()

<br>

In [ ]:
# scatterplot - orig ovr_qual and log price
sns.scatterplot(x    = 'Overall_Qual',
                y    = 'log_Sale_Price',
                data = housing)


# titles and axis labels
plt.title(label   = 'Scatterplot with Interval Data')
plt.xlabel(xlabel = 'Overall Quality')
plt.ylabel(ylabel = 'Sale Price')


# displaying the plot
plt.show()

In [ ]:
# boxplot - ori ovr_qual and log price
sns.boxplot(x    = 'Overall_Qual',
            y    = 'log_Sale_Price',
            data = housing)


# titles and axis labels
plt.title(label   = 'Boxplot with Interval Data')
plt.xlabel(xlabel = 'Overall Quality')
plt.ylabel(ylabel = 'Sale Price')


# displaying the plot
plt.show()

<br>

In [ ]:
# correlations - ovr qual and prices
log_corr = housing.loc[ : , ['Overall_Qual',
                             'Sale_Price',
                             'log_Sale_Price']  ].corr(method = 'pearson')\
                                                 .round(decimals = 2)


# checking results
log_corr.loc[ ['Overall_Qual'],
              ['Sale_Price', 'log_Sale_Price']   ]

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br><br>
<strong>Tips to Analyzing Relationships with Boxplots</strong><br>

* Start by analyzing differences in the median lines for each boxplot. Does it appear that the median lines can be connected using a straight horizontal line? If not, move to the next bullet point.
* Engineer features as necessary. Make sure you have at least 100 observations in each part of your newly-engineered feature(s).

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part III: Feature Engineering with Count Data</h2><br>
Let's take a look at the relationship between the number of bathrooms (full baths and half baths) and sale price.

In [ ]:
# setting figure size
fig, ax = plt.subplots(figsize = (15, 10))


## full baths ##

# first plot window
plt.subplot(1, 2, 1)

# developing a boxplot
sns.boxplot(x     = 'Full_Bath',
            y     = 'Sale_Price',
            data  = housing)


# titles and labels
plt.title(label   = 'Relationship between Full Bathrooms and Sale Price')
plt.xlabel(xlabel = 'Full Bathrooms')
plt.ylabel(ylabel = 'Sale Price')



## half baths ##

# second plot window
plt.subplot(1, 2, 2)

# developing a boxplot
sns.boxplot(x     = 'Half_Bath',
            y     = 'Sale_Price',
            data  = housing)


# titles and labels
plt.title(label   = 'Relationship between Half Bathrooms and Sale Price')
plt.xlabel(xlabel = 'Half Bathrooms')
plt.ylabel(ylabel = 'Sale Price')


# displaying the plot
plt.tight_layout
plt.show()


# printing value counts for full and half baths
print(f"""
Count of Bathrooms
==================

----------
Full Baths
----------
{housing['Full_Bath'].value_counts(normalize = False).sort_index()}


----------
Half Baths
----------
{housing['Half_Bath'].value_counts(normalize = False).sort_index()}


""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<strong>Combining Features</strong><br>
Now, let's engineer a feature that represents total bathrooms using the following calculation:
<br><br>

~~~
Total_Baths = Full_Baths + (Half_Baths / 2)
~~~

<br>
<strong>a)</strong> Complete the code below to engineer Total_Baths based on the calculation above and check your results.

In [ ]:
# total bathrooms
housing['Total_Bath'] = ____


# setting figure size
fig, ax = plt.subplots(figsize = (15, 10))


# developing a boxplot
sns.boxplot(x    = ____,
            y    = ____,
            data = ____)



# titles and labels
plt.title(label   = 'Relationship between Total Bathrooms and Sale Price')
plt.xlabel(xlabel = 'Total Bathrooms')
plt.ylabel(ylabel = 'Sale Price')


# displaying the plot
plt.tight_layout
plt.show()


## the following code has been written for you ##

# developing a correlation matrix
bath_corr = housing.loc[ : , ['Full_Bath',
                              'Half_Bath',
                              'Total_Bath',
                              'Sale_Price']  ].corr(method = 'pearson')\
                                              .round(decimals = 2)


# printing value counts and correlations bathrooms
print(f"""
Bathrooms
=========

-----------
Total Baths
-----------
{housing['Total_Bath'].value_counts(normalize = False).sort_index()}


---------------------
Bathroom Correlations
---------------------
{bath_corr['Sale_Price']}

""")

In [ ]:
# total bathrooms
housing['Total_Bath'] = housing['Full_Bath'] + (housing['Half_Bath'] / 2)


# setting figure size
fig, ax = plt.subplots(figsize = (15, 10))


# developing a boxplot
sns.boxplot(x    = 'Total_Bath',
            y    = 'Sale_Price',
            data = housing)



# titles and labels
plt.title(label   = 'Relationship between Total Bathrooms and Sale Price')
plt.xlabel(xlabel = 'Total Bathrooms')
plt.ylabel(ylabel = 'Sale Price')


# displaying the plot
plt.tight_layout
plt.show()


# developing a correlation matrix
bath_corr = housing.loc[ : , ['Full_Bath',
                              'Half_Bath',
                              'Total_Bath',
                              'Sale_Price']  ].corr(method = 'pearson')\
                                             .round(decimals = 2)


# printing value counts and correlations bathrooms
print(f"""
Bathrooms
=========

----------
Total Bath
----------
{housing['Total_Bath'].value_counts(normalize = False).sort_index()}


---------------------
Bathroom Correlations
---------------------
{bath_corr['Sale_Price']}

""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part IV: Feature Engineering with Categorical Data</h2><br>
Categorical features need to be converted into a series of yes or no questions, more formally known as <strong>binary matrix factorization</strong>. One category needs to be removed to avoid issues with autocorrelation (the <em>k-1</em> rule).

In [ ]:
# importing interval and count data
file = './datasets/ames_non_continuous.xlsx'

housing_2 = pd.read_excel(io         = file,
                          header     = 0,
                          sheet_name = 'categorical_discrete')


# documentation: help(pd.DataFrame.merge)

# merging the two datasets
housing = housing.merge(right = housing_2,
                        on    = 'property_id',
                        how   = 'left')


# checking results
housing.head(n = 5)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Run the following code to see the value counts for each categorical feature.

In [ ]:
# printing columns
print(f"""
Street
------
{housing['Street'].value_counts()}


Lot Config
----------
{housing['Lot_Config'].value_counts()}


Neighborhood
-------------
{housing['Neighborhood'].value_counts()}
""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>The user-defined function below will generate boxplots for categorical data.</h4>

In [ ]:
# defining a function for categorical boxplots
def categorical_boxplots(response, cat_var, data):
    """
	This function is designed to generate a boxplot for  can be used for categorical variables.
    Make sure matplotlib.pyplot and seaborn have been imported (as plt and sns).

    PARAMETERS
	----------
	response : str, response variable
	cat_var  : str, categorical variable
	data     : DataFrame of the response and categorical variables
	"""

    fig, ax = plt.subplots(figsize = (10, 8))
    
    sns.boxplot(x    = response,
                y    = cat_var,
                data = data)
    
    plt.suptitle("")
    plt.show()

<br>

In [ ]:
# calling the function for Street
categorical_boxplots(response = 'Sale_Price',
					 cat_var  = 'Street',
					 data     = housing)


# calling the function for Lot_Config
categorical_boxplots(response = 'Sale_Price',
					 cat_var  = 'Lot_Config',
					 data     = housing)


# calling the function for Neighborhood
categorical_boxplots(response = 'Sale_Price',
					 cat_var  = 'Neighborhood',
					 data     = housing)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<strong>a)</strong> Fill in the blanks below to one hot encode each categorical feature using <em>get_dummies(&nbsp;)</em>. Then, remove the original categorical features using <em>drop(&nbsp;)</em>. Finally, merge the one hot encoded data with the original dataset using <em>join(&nbsp;)</em>. Note that running the following code more than once will generate an error because it is dropping the original categorical features.

In [ ]:
# one hot encoding categorical variables
one_hot_Lot          = pd._____(housing['Lot_Config'])
one_hot_Neighborhood = pd._____(housing['Neighborhood'])


# dropping categorical variables after they've been encoded
housing = housing._____(columns = ['Street', 'Lot_Config', 'Neighborhood'],
                        axis = 1,
                        dtype = int)


# joining codings together
housing = housing.join([one_hot_Lot, one_hot_Neighborhood])


# saving new columns
new_columns = housing.columns

In [ ]:
# one hot encoding categorical variables
one_hot_Lot          = pd.get_dummies(housing['Lot_Config'],   dtype = int)
one_hot_Neighborhood = pd.get_dummies(housing['Neighborhood'], dtype = int)


# dropping categorical variables after they've been encoded
housing = housing.drop('Street',       axis = 1) # dropping even though not encoded
housing = housing.drop('Lot_Config',   axis = 1)
housing = housing.drop('Neighborhood', axis = 1)


# joining codings together
housing = housing.join([one_hot_Lot, one_hot_Neighborhood])

<br>

In [ ]:
# checking results
housing.head(n = 5)

<br>

In [ ]:
## this will convert Lot_Config into CulDSac Y/N ##

# dropping categories that will not be used
housing = housing.drop( ["Corner", "FR2", "FR3", "Inside"], axis = 1)

<br>

In [ ]:
# combining smaller sample sizes into "Other" group
housing["Other_NH"] = housing.loc[ : ,  ["IDOTRR", "Timber", "NoRidge", "StoneBr",
                                         "SWISU", "ClearCr", "MeadowV", "BrDale",
                                         "Blmngtn", "Veenker", "NPkVill", "Blueste",
                                         "Greens", "GrnHill", "Landmrk"]  ].sum(axis = 1)


housing = housing.drop(["IDOTRR", "Timber", "NoRidge", "StoneBr",
                        "SWISU", "ClearCr", "MeadowV", "BrDale",
                        "Blmngtn", "Veenker", "NPkVill", "Blueste",
                        "Greens", "GrnHill", "Landmrk"], axis = 1)


# checking results
housing["Other_NH"].value_counts(normalize = True).round(decimals = 2)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part V: Building Predictive Models</h2>
<br>
What we're about to do is considered data dumping. <font color="red">You will lose points on your assignments for doing something like this.</font>

In [ ]:
# instantiating a correlation matrix
df_corr = housing.corr().round(2)


# setting figure size
fig, ax = plt.subplots(figsize=(15,15))


# visualizing the correlation matrix
sns.heatmap(df_corr,
            cmap = 'coolwarm',
            square = True,
            annot = True,
            linecolor = 'black',
            linewidths = 0.5)


# layout and rendering
plt.tight_layout()
plt.show()

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

It will be more efficient to simply analyze the Pearson correlations with the Y-variable(s), as in the code below.

In [ ]:
# creating a (Pearson) correlation matrix
df_corr = housing.corr().round(2)


# printing (Pearson) correlations with SalePrice
df_corr.loc[ : , ['Sale_Price', 'log_Sale_Price'] ].sort_values(by = 'Sale_Price',
                                                                ascending = False)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<strong>b)</strong> Sale_Price or log_Sale_Price?
<br>    
Write a code that:

 * takes the results of the correlation output above
 * determines the higher correlation (<em>Sale_Price</em> or <em>log_Sale_Price</em>)
 * outputs the results as a frequency table

In [ ]:
# write your code here





In [ ]:
# subsetting for sale prices
price_corr = df_corr.loc[ : , ['Sale_Price', 'log_Sale_Price'] ].sort_values(by = 'Sale_Price',
                                                                 ascending = False)


# removing irrelevant correlations
price_corr = price_corr.iloc[ 2: , : ]


# placeholder column for y-variable recommendation
price_corr['original_v_log'] = 0


# filling in placeholder
for index, column in price_corr.iterrows():
    
    # if Sale_Price is higher
    if abs(price_corr.loc[ index, 'Sale_Price']) >  abs(price_corr.loc[ index, 'log_Sale_Price']):
        price_corr.loc[ index , 'original_v_log'] = 'Sale_Price'
        
        
    # if log_Sale_Price is higher 
    elif abs(price_corr.loc[ index, 'Sale_Price']) <  abs(price_corr.loc[ index, 'log_Sale_Price']):
        price_corr.loc[ index , 'original_v_log'] = 'log_Sale_Price'
    
    
    # in case they are tied
    else:
        price_corr.loc[ index , 'original_v_log'] = 'Tie'
        

# checking results
price_corr["original_v_log"].value_counts(normalize = False,
                                          sort      = True,
                                          ascending = False).round(decimals = 2)

### <hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Valuable (Pearson) correlations have been discovered from many of the newly engineered-features. Let's throw everything into a model and see how each explanatory variable performs. This is known as building a <strong>full model</strong>.<br><br>
The following code is meant to drop the Y-variables (&nbsp;<em>Sale_Price</em> and <em>log_Sale_Price</em>) as well as <em>property_id</em>. This is so that the Y-variable doesn't accidentally end up on the X-side of the models we develop. Remember, <font color = "red"> the Y-variable belongs on the Y-side of the model. Putting it on the X-side in any form, such as through newly-engineered features is an invalid approach to modeling.</font> This is because if we are truly interested in predicting Y, we will not have its data at the time of prediction (Y will be unknown).<br><br>

<strong>c)</strong> Adding each variable one-by-one would be a long and tedious process. Perform these steps to be more efficient:

1. Make a copy of the housing dataset and name it <em>housing_explanatory</em>
2. Drop <em>Sale_Price</em>, <em>log_Sale_Price</em>, and <em>property_id</em> from <em>housing_explanatory</em>
3. Develop a loop that will print each explanatory variable in the following format:<br>

~~~
EXPLANATORY_VARIABLE +
~~~

In [ ]:
# making a copy of housing
housing_explanatory = _____


# dropping SalePrice and Order from the explanatory variable set
housing_explanatory = ____.____(['Sale_Price',
                                 'log_Sale_Price',
                                 'property_id'], axis = 1)


# formatting each explanatory variable for statsmodels
for val in housing_explanatory:
    _____

In [ ]:
# making a copy of housing
housing_explanatory = housing.copy()


# dropping Sale_Price, log_Sale_Price, and property_id from the explanatory variable set
housing_explanatory = housing_explanatory.drop(['Sale_Price',
                                                'log_Sale_Price',
                                                'property_id'], axis = 1)


# formatting each explanatory variable for statsmodels
for val in housing_explanatory:
    print(f"{val} +")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<strong>d)</strong> Copy/Paste the results of the loop above into the following regression modeling template.<br><br>
Then, modify the results to create a predictive model using the full model.

In [ ]:
# building a full model

# blueprinting a model type
lm_full = smf.ols(formula = """INSERT COPY/PASTE HERE""",
                               data = housing)


# telling Python to run the data through the blueprint
results_full = lm_full.fit()


# printing the results
results_full.summary()

In [ ]:
# building a full model

# blueprinting a model type
lm_full = smf.ols(formula = """Sale_Price ~ Lot_Area +
                                            Mas_Vnr_Area +
                                            Total_Bsmt_SF +
                                            First_Flr_SF +
                                            Second_Flr_SF +
                                            Gr_Liv_Area +
                                            Garage_Area +
                                            Porch_Area +
                                            m_Lot_Area +
                                            m_Mas_Vnr_Area +
                                            m_Second_Flr_SF +
                                            log_Lot_Area +
                                            log_Gr_Liv_Area +
                                            has_Second_Flr +
                                            has_Garage +
                                            has_Mas_Vnr +
                                            has_Porch +
                                            ratio_building_lot +
                                            Overall_Qual +
                                            Overall_Cond +
                                            Full_Bath +
                                            Half_Bath +
                                            Kitchen_AbvGr +
                                            TotRms_AbvGr +
                                            Fireplaces +
                                            Garage_Cars +
                                            Total_Bath +
                                            CulDSac +
                                            BrkSide +
                                            CollgCr +
                                            Crawfor +
                                            Edwards +
                                            Gilbert +
                                            Mitchel +
                                            NAmes +
                                            NWAmes +
                                            NridgHt +
                                            OldTown +
                                            Sawyer +
                                            SawyerW +
                                            Somerst +
                                            Other_NH
                                            """,
                                data = housing)

                        

# telling Python to run the data through the blueprint
results_full = lm_full.fit()


# printing the results
results_full.summary()

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<strong>d)</strong> Create your best model by removing insignificant features based on their p-values.<br><br>
This is known as developing a <strong>fit model</strong>. Note that as soon as a feature is removed from the model, other p-values will change due to <strong>collinearity</strong> (correlation amongst explanatory variables). Adjusted R-Square is your metric for success as it takes into account the size of the model (penalizing larger models that do not create additional predictive value).
<br><br><br>
<strong>Goals</strong>

* Adjusted R-square of 0.825 or higher
* All p-values in model $\le$ 0.05

In [ ]:
# write your code here





In [ ]:
## Model Example 1 (Sale_Price) ##

# building a fit model

# blueprinting a model type
lm_fit = smf.ols(formula = """Sale_Price ~ Garage_Cars +
                                           Overall_Qual +
                                           Total_Bsmt_SF +
                                           NridgHt +
                                           Kitchen_AbvGr +
                                           has_Second_Flr +
                                           Mas_Vnr_Area +
                                           has_Garage +
                                           Porch_Area +
                                           NWAmes +
                                           OldTown +
                                           Overall_Cond +
                                           NAmes +
                                           Edwards +
                                           Lot_Area +
                                           Somerst +
                                           Fireplaces +
                                           Second_Flr_SF +
                                           First_Flr_SF +
                                           has_Mas_Vnr +
                                           CulDSac +
                                           Total_Bath +
                                           Crawfor +
                                           Garage_Area +
                                           has_Porch""",
                                data = housing)


# telling Python to run the data through the blueprint
results_fit = lm_fit.fit()


# printing the results
results_fit.summary()

<br>

In [ ]:
## Model Example 2 (log_Sale_Price) ##

# building a fit model

# blueprinting a model type
lm_fit = smf.ols(formula = """log_Sale_Price ~ Garage_Cars +
                                               Overall_Qual +
                                               Total_Bsmt_SF +
                                               OldTown +
                                               Overall_Cond +
                                               Kitchen_AbvGr +
                                               Total_Bath +
                                               has_Second_Flr +
                                               NridgHt +
                                               Fireplaces +
                                               Porch_Area +
                                               Somerst +
                                               CollgCr +
                                               Lot_Area +
                                               Crawfor +
                                               CulDSac +
                                               NWAmes +
                                               Second_Flr_SF +
                                               First_Flr_SF +
                                               Edwards +
                                               Gilbert""",
                                               data = housing)


# telling Python to run the data through the blueprint
results_fit = lm_fit.fit()


# printing the results
results_fit.summary()

<br><br><hr style="height:.9px;border:none;color:#333;background-color:#333;" /><hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

~~~
 ___  ___      ___       __   ___    
|__  |__   /\   |  |  | |__) |__  __ 
|    |___ /~~\  |  \__/ |  \ |___    
                                     
 __   __     ___  ___    __          
|__) |__) | |__  |__  | /  `         
|  \ |  \ | |    |    | \__,         
                                                                    
~~~

<br>
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<br>